# Lab : LightGBMのカスタムコンテナを通して、SageMakerの動作を理解する

LightGBMがインストールされたカスタムコンテナを構築し、SageMaker Trainingジョブで学習を行います。
カスタムコンテナの挙動を観察し、SageMakerの動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [ ]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

In [ ]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [ ]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [ ]:
import sklearn
sklearn.__version__ # 1.0.1

In [ ]:
import pandas as pd
pd.__version__ # 1.3.4

## 1-1. データロード

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [ ]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [ ]:
# 確認
trainX.head()

In [ ]:
# 確認
valX.head()

In [ ]:
# 確認
testX.head()

In [ ]:
# 確認
y_test[0:5]

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [ ]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [ ]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [ ]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [ ]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

# 2.LightGBMカスタムコンテナの構築


カスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

SageMakerの動作を理解するためにパターン3のベースイメージ + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

In [ ]:
!pygmentize ./container/Dockerfile

## 2-2. dockerイメージの build & push

ビルド&pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
ECRでpushしたコンテナのURIを確認

AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを取得し、以下にはりつけます。

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [ ]:
# 確認
image_uri

In [ ]:
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [ ]:
from sagemaker.estimator import Estimator

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
# 確認
role

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [ ]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果は

Amazon S3
Buckets
sagemaker-us-west-2-805433377179
sagemaker-lightgbm-regression-2022-10-03-06-17-32-054/

に出力されます。


## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
!docker ps

In [ ]:
# 起動中のコンテナを停止する場合
#!docker stop XXXXXXXXXXX #XXXXXXXXXXXは CONtAINER ID
#!docker ps

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [ ]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3. 実行ファイルを外部から指定する

「2.LightGBMカスタムコンテナの構築」ではカスタムコンテナ内に学習起動スクリプトtrainを配置しましたが、
ソースコードを修正するごとにコンテナを作り替える必要があります。

保守性を上げるには、コンテナ（環境）とソースコードを分けた方がいい場合もあります。
以下では外部からスクリプトファイルを指定する方法を紹介します。

## 3-0.SageMaker Training Toolkitとは
外部からスクリプトを指定するためには、SageMaker Training Toolkitを導入します。

https://github.com/aws/sagemaker-training-toolkit


trainコマンドが  
/opt/conca/bin/train  
にインストールされます。  


先程のdockerfileに追記します。
資材からは、trainを除外しておきます。trainを含んだままの場合、
docker run <image> train
を実行したときに、カレントディレクトリのtrainスクリプトが実行されてしまい、training toolkitが導入した　trainコマンドが実行できないためです。

## 3-1. Dockerfile確認

In [ ]:
!pygmentize ./container_smtrtoolkit/Dockerfile

## 3-2. build & push

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-toolkit

#cd container
cd container_smtrtoolkit ### 変更点

#chmod +x lightgbm_regression/train ### 変更点
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


## 3-3.学習(ローカル)

In [ ]:
image_uri_toolkit = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-toolkit'

In [ ]:
# 確認
image_uri_toolkit

In [ ]:
est_lightgbm_toolkit = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    entry_point='./src/train_practice.py'
    )

In [ ]:
est_lightgbm_toolkit.fit({'train':train_s3, 'validation': valid_s3})

ローカルモードで学習することができました。別のスクリプトを指定してみましょう。

In [ ]:
est_lightgbm_toolkit2 = Estimator(
    image_uri_toolkit,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    #entry_point='./src/train_practice.py'
    entry_point='./src/train_practice.sh' ### シェルスクリプトに変更
    )

In [ ]:
est_lightgbm_toolkit2.fit({'train':train_s3, 'validation': valid_s3})

コンテナ外部から任意のファイルを実行することが確認できました。

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


In [ ]:
container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [ ]:
container_uri

In [ ]:
est_xgb = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    source_dir='./src_builtin_container',
    entry_point='train_practice.py'
    )

In [ ]:
est_xgb.fit({'train':train_s3, 'validation': valid_s3})

## 4-2. requirements.txtが存在しない場合エラーになることを確認する

In [ ]:
est_xgb_no_lgbm = Estimator(
    container_uri, # xgboostのbuilt-inコンテナ
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters,
    source_dir='./src_builtin_container_no_lgbm',
    entry_point='train_practice.py'
    )

In [ ]:
est_xgb_no_lgbm.fit({'train':train_s3, 'validation': valid_s3})

lightgbmモジュールが存在しないため、エラーとなります

File "/opt/ml/code/train_practice.py", line 13, in <module>  
import lightgbm as lgb  
ModuleNotFoundError: No module named 'lightgbm'  


# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit